In [ ]:
import csv
import datetime
import glob
import os
import re
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import quandl as q
import streamlit as st
import requests
import json
import time
import warnings
from astral import moon
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

import requests
import json
from dateutil import tz
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from fredapi import Fred

api_key_fred = os.environ["api_key_fred"]
fred = Fred(api_key=api_key_fred)

quandl_api_key = os.environ["quandl_api_key"]

In [ ]:

# # # start - read in BINANCE BTC data # # # binance only goes back to 2017!
def get_binance_crypto_usdt(selected_crypto : str = "BTC", selected_interval : str = "1d"):
    URL = "https://api.binance.com/api/v3/klines"
    start_str = "2020-01-01 00:00:00"
    fmt = "%Y-%m-%d %H:%M:%S"
    start_time = int(time.mktime(time.strptime(start_str, fmt)) * 1000)
    last_open_time = 0  # added this line

    df = pd.DataFrame()

    while True:
        parameters = {
            "symbol": f"{selected_crypto}USDT",
            "interval": selected_interval,
            "startTime": start_time,
            "limit": 1000,  # maximum limit
        }

        response = requests.get(URL, params=parameters)
        data = json.loads(response.text)

        if len(data) < 1 or last_open_time == start_time:  # updated this line
            break

        temp_df = pd.DataFrame(
            data,
            columns=[
                "Open_time",
                "Open",
                "High",
                "Low",
                "Close",
                "Volume",
                "Close_time",
                "Quote_asset_volume",
                "Number_of_trades",
                "Taker_buy_base",
                "Taker_buy_quote",
                "Ignore",
            ],
        )

        temp_df["Open_time"] = pd.to_datetime(temp_df["Open_time"], unit="ms")
        temp_df["Date"] = temp_df["Open_time"].dt.date
        temp_df["High"] = temp_df["High"].astype(float)
        temp_df["Low"] = temp_df["Low"].astype(float)
        temp_df["Last"] = temp_df["Close"].astype(float)
        temp_df["Volume"] = temp_df["Volume"].astype(float)
        temp_df["Mid"] = (temp_df["High"] + temp_df["Low"]) / 2
        temp_df["First"] = temp_df["Last"].shift()

        print(temp_df)

        df = pd.concat([df, temp_df])

        if not temp_df.empty:
            last_open_time = (
                start_time  # This line should only run if temp_df is not empty.
            )
            start_time = (
                int(temp_df["Open_time"].dt.to_pydatetime()[-1].timestamp() * 1000) + 1
            )
        else:
            break  # If temp_df is empty, we should break from the loop.

    if not df.empty:
        df = df[["Date", "Open_time", "High", "Low", "Mid", "Last", "Volume", "First"]]
        df.to_csv(f"coindata/binance {selected_crypto}USDT.csv", index=False)
    else:
        print(f"No data collected. API response {response}")


cryptos = [
    "BTC",
    "ETH",
    "DOGE",
    "LINK",
    "OP",
    "MATIC",
    "XRP",
    "LTC",
    "EOS",
    "MANA",
    "SAND",
]

# selected_crypto = st.selectbox("Select Cryptocurrency", cryptos)
selected_crypto = "BTC"


datasource = f"binance/{selected_crypto}USDT.csv"

todays_date = datetime.date.today()
todays_date = todays_date.strftime("%Y-%m-%d")

try:
    xusd_data = pd.read_csv(
        "coindata/{}".format(datasource.replace("/", " ")), index_col=0
    )
except:
    get_binance_crypto_usdt(selected_crypto)
    xusd_data = pd.read_csv(
        "coindata/{}".format(datasource.replace("/", " ")), index_col=0
    )


datasource = f"binance {selected_crypto}USDT.csv"
xusd_data = pd.read_csv("coindata/{}".format(datasource), index_col=0)
xusd_data.index = pd.to_datetime(xusd_data.index)

In [ ]:
# get_binance_crypto_usdt(selected_crypto, "1h")

              Open_time           Open      High      Low           Close  \
0   2019-12-31 23:00:00  7200.52000000   7206.29  7185.76   7195.23000000   
1   2020-01-01 00:00:00  7195.24000000   7196.25  7175.46   7177.02000000   
2   2020-01-01 01:00:00  7176.47000000   7230.00  7175.71   7216.27000000   
3   2020-01-01 02:00:00  7215.52000000   7244.87  7211.41   7242.85000000   
4   2020-01-01 03:00:00  7242.66000000   7245.00  7220.00   7225.01000000   
..                  ...            ...       ...      ...             ...   
995 2020-02-11 11:00:00  9786.89000000   9815.00  9775.07   9801.04000000   
996 2020-02-11 12:00:00  9801.02000000   9877.54  9787.88   9847.74000000   
997 2020-02-11 13:00:00  9847.74000000   9908.00  9846.46   9871.43000000   
998 2020-02-11 14:00:00  9871.43000000   9901.50  9847.00   9857.95000000   
999 2020-02-11 15:00:00  9857.19000000  10194.00  9852.58  10164.99000000   

           Volume     Close_time  Quote_asset_volume  Number_of_trades  \
0

In [ ]:
xusd_data = pd.read_csv(
        'coindata/binance BTCUSDT 1h.csv', index_col=0
    )
xusd_data.head()

,Open_time,High,Low,Mid,Last,Volume,First
Date,,,,,,,
2019-12-31,2019-12-31 23:00:00,7206.29,7185.76,7196.025,7195.23,521.978397,NaN
2020-01-01,2020-01-01 00:00:00,7196.25,7175.46,7185.855,7177.02,511.814901,7195.23
2020-01-01,2020-01-01 01:00:00,7230.00,7175.71,7202.855,7216.27,883.052603,7177.02
2020-01-01,2020-01-01 02:00:00,7244.87,7211.41,7228.140,7242.85,655.156809,7216.27
2020-01-01,2020-01-01 03:00:00,7245.00,7220.00,7232.500,7225.01,783.724867,7242.85


In [ ]:
most_recent_stored_btcusd_date = (
    xusd_data.sort_index().tail(1).index[0].strftime("%Y-%m-%d")
)
most_recent_stored_btcusd_date

AttributeError: 'str' object has no attribute 'strftime'

In [ ]:

if most_recent_stored_btcusd_date != todays_date:
    get_binance_crypto_usdt(selected_crypto)

    xusd_data = pd.read_csv("coindata/{}".format(datasource), index_col=0)
    xusd_data.index = pd.to_datetime(xusd_data.index)

# # # # end - read in BINANCE BTC data # # #

In [ ]:
# # # start - data processing # # #
xusd_data = xusd_data.dropna()
xusd_data["350_movingaverage"] = pd.Series.rolling(
    xusd_data["Last"], window=350, min_periods=1
).mean()
xusd_data["111_movingaverage"] = pd.Series.rolling(
    xusd_data["Last"], window=111, min_periods=1
).mean()
# # # end - data processing # # #

C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\3808707260.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xusd_data["350_movingaverage"] = pd.Series.rolling(
C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\3808707260.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xusd_data["111_movingaverage"] = pd.Series.rolling(


In [ ]:
xusd_data

,High,Low,Mid,Last,Volume,First,350_movingaverage,111_movingaverage
Date,,,,,,,,
2017-08-18,4371.52,3938.77,4155.145,4108.37,1199.888264,4285.08,4108.370000,4108.370000
2017-08-19,4184.69,3850.00,4017.345,4139.98,381.309763,4108.37,4124.175000,4124.175000
2017-08-20,4211.08,4032.62,4121.850,4086.29,467.083022,4139.98,4111.546667,4111.546667
2017-08-21,4119.62,3911.79,4015.705,4016.00,691.743060,4086.29,4087.660000,4087.660000
2017-08-22,4104.82,3400.00,3752.410,4040.00,966.684858,4016.00,4078.128000,4078.128000
...,...,...,...,...,...,...,...,...
2024-03-19,68124.11,61555.00,64839.555,61937.40,101005.324870,67609.99,35473.083400,48753.019369
2024-03-20,68100.00,60775.00,64437.500,67840.51,90420.585920,61937.40,35586.427686,49024.339640
2024-03-21,68240.47,64529.01,66384.740,65501.27,53357.480020,67840.51,35693.477543,49265.950000


In [ ]:
def calculate_emas(df, column_name="Last", days=[12, 25, 200, 700, 1400]):
    """
    Calculate Exponential Moving Averages (EMAs) for specified days.
    
    Parameters:
    - df: pandas DataFrame with a date index and a column for prices.
    - column_name: String name of the column containing price data.
    - days: List of integers for which to calculate EMAs.
    
    Returns:
    - A DataFrame with original data and additional columns for each EMA.
    """
    for day in days:
        ema_column_name = f"EMA_{day}"
        df[ema_column_name] = df[column_name].ewm(span=day, adjust=False).mean()
    
    return df

calculate_emas(xusd_data)

C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\1576874728.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\1576874728.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\1576874728.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

,High,Low,Mid,Last,Volume,First,350_movingaverage,111_movingaverage,EMA_12,EMA_25,Trigger,EMA_200,EMA_700,EMA_1400
Date,,,,,,,,,,,,,,
2017-08-18,4371.52,3938.77,4155.145,4108.37,1199.888264,4285.08,4108.370000,4108.370000,4108.370000,4108.370000,None,4108.370000,4108.370000,4108.370000
2017-08-19,4184.69,3850.00,4017.345,4139.98,381.309763,4108.37,4124.175000,4124.175000,4113.233077,4110.801538,BULL,4108.684527,4108.460185,4108.415125
2017-08-20,4211.08,4032.62,4121.850,4086.29,467.083022,4139.98,4111.546667,4111.546667,4109.087988,4108.916036,None,4108.461696,4108.396932,4108.383540
2017-08-21,4119.62,3911.79,4015.705,4016.00,691.743060,4086.29,4087.660000,4087.660000,4094.766759,4101.768648,BEAR,4107.541679,4108.133318,4108.251658
2017-08-22,4104.82,3400.00,3752.410,4040.00,966.684858,4016.00,4078.128000,4078.128000,4086.341104,4097.017214,None,4106.869623,4107.938929,4108.154225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-19,68124.11,61555.00,64839.555,61937.40,101005.324870,67609.99,35473.083400,48753.019369,67002.556073,64175.970123,None,44337.818885,34261.252057,29750.173657
2024-03-20,68100.00,60775.00,64437.500,67840.51,90420.585920,61937.40,35586.427686,49024.339640,67131.472062,64457.857806,None,44571.676508,34357.055932,29804.549584
2024-03-21,68240.47,64529.01,66384.740,65501.27,53357.480020,67840.51,35693.477543,49265.950000,66880.671745,64538.120282,None,44779.931170,34445.912463,29855.508499


In [ ]:
def update_triggers(df):
    # Eine neue Spalte für Triggers hinzufügen, initialisiert mit None
    df['Trigger'] = None

    for i in range(1, len(df)):
        if df['EMA_12'].iloc[i] > df['EMA_25'].iloc[i] and df['EMA_12'].iloc[i-1] <= df['EMA_25'].iloc[i-1]:
            df['Trigger'].iloc[i] = 'BULL'
        elif df['EMA_12'].iloc[i] < df['EMA_25'].iloc[i] and df['EMA_12'].iloc[i-1] >= df['EMA_25'].iloc[i-1]:
            df['Trigger'].iloc[i] = 'BEAR'

update_triggers(xusd_data)

C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\718427680.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\718427680.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_21000\718427680.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:

# diagram - ema 12 vs ema 25
fig = go.Figure(
    data=go.Scatter(
        x=xusd_data.index,
        y=xusd_data["Last"],
        mode="lines",
        marker=dict(
            # size=16,
            color="black",  # set color equal to a variable
        ),
    )
)
fig.add_trace(
    go.Scatter(
        x=xusd_data.index,
        y=xusd_data["EMA_12"],
        mode="lines",
        name="EMA_12",
        marker=dict(
            color="cyan"
        ),	
    )
)
fig.add_trace(
    go.Scatter(
        x=xusd_data.index,
        y=xusd_data["EMA_25"],
        mode="lines",
        name="EMA_25",
        marker=dict(
            color="blue"
        ),
    )
)

fig.add_trace(
    go.Scatter(
        x=xusd_data.index,
        y=xusd_data["EMA_200"],
        mode="lines",
        name="EMA_200",
        marker=dict(
            color="gold"
        ),
    )
)

fig.add_trace(
    go.Scatter(
        x=xusd_data.index,
        y=xusd_data["EMA_700"],
        mode="lines",
        name="weekly 100",
        marker=dict(
            color="gray"
        ),
    )
)

fig.add_trace(
    go.Scatter(
        x=xusd_data.index,
        y=xusd_data["EMA_1400"],
        mode="lines",
        name="weekly 200",
        marker=dict(
            color="black"
        ),
    )
)

# Hinzufügen von Markierungen für "BULL" Ereignisse
bull_indices = xusd_data[xusd_data['Trigger'] == 'BULL'].index
fig.add_trace(
    go.Scatter(
        x=xusd_data.loc[bull_indices].index,
        y=xusd_data.loc[bull_indices]['Last'],
        mode='markers',
        name='BULL',
        marker=dict(
            color='green',
            size=14,
            symbol='triangle-up'
        ),
    )
)

# Hinzufügen von Markierungen für "BEAR" Ereignisse
bear_indices = xusd_data[xusd_data['Trigger'] == 'BEAR'].index
fig.add_trace(
    go.Scatter(
        x=xusd_data.loc[bear_indices].index,
        y=xusd_data.loc[bear_indices]['Last'],
        mode='markers',
        name='BEAR',
        marker=dict(
            color='red',
            size=14,
            symbol='triangle-down'
        ),
    )
)

fig.update_yaxes(type="log")
fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400 / 1.1),
    height=int(800 / 1.1),
    title=f"Pi Cycle Top Indicator {selected_crypto}/USD",
)
fig.show()


In [ ]:
def check_and_notify(df):
    # Die neueste Zeile (basierend auf dem Index, der das Datum sein sollte) abrufen
    latest_row = df.iloc[-1]
    
    # Überprüfen, ob ein Trigger in der neuesten Zeile vorhanden ist
    if pd.notnull(latest_row['Trigger']):
        # Nachricht senden (hier durch Print simuliert)
        print(f"Nachricht: {latest_row['Trigger']} Trigger am {df.index[-1].date()}")
    else:
        print("Kein Trigger heute.")

check_and_notify(xusd_data)  # Rufen Sie diese Funktion auf, um die neueste Zeile zu überprüfen und ggf. eine Nachricht zu senden.


# TODO
# ### 1. Datenerfassung
# - **Stündliches Scraping von Kryptowährungsdaten**
#   - Auswahl der Coins, von denen Daten gesammelt werden sollen.
#   - Entwicklung eines Scraping-Scripts, das stündlich läuft.
#   - Sicherstellung der Datenspeicherung in einer geeigneten Datenbank oder einem Dateiformat.

# ### 2. Backtesting der Strategie
# - **Entwicklung des Backtesting-Frameworks**
#   - Implementierung eines Backtesting-Systems, das verschiedene Zeitintervalle unterstützt (täglich, stündlich, vierstündlich, 12-stündlich).
#   - Definition der Handelsstrategie und der Regeln für Ein- und Ausstieg.
#   - Durchführung von Backtests der Strategie über historische Daten, um die Performance zu evaluieren.

# ### 3. Automatisierung und Scheduling
# - **Einrichtung eines Automatisierungsscripts**
#   - Entscheidung über die Frequenz der Strategieausführung (täglich, stündlich etc.).
#   - Nutzung eines Schedulers (z.B. cron für Linux/Mac oder Task Scheduler für Windows) für die Automatisierung des Scripts.
#   - Überlegungen zur Nutzung eines Servers (z.B. Oracle Cloud) für kontinuierlichen Betrieb.

# ### 4. Benachrichtigungssystem
# - **Implementierung eines Benachrichtigungssystems**
#   - Entwicklung eines Systems, das bei bestimmten Ereignissen oder Bedingungen Benachrichtigungen versendet, z.B. per E-Mail.
#   - Konfiguration des Systems zur automatischen Benachrichtigung bei wichtigen Handelssignalen oder Systemfehlern.

# ### 5. Integration von Live-Trading
# - **Aufbau der Live-Trading-Funktion**
#   - Auswahl einer Trading-Plattform (z.B. Bybit) für den automatisierten Handel.
#   - Implementierung der API-Anbindung zur Ausführung von Trades basierend auf den Signalen der Handelsstrategie.
#   - Einrichtung von Sicherheitsmaßnahmen und Limitierungen (STOP LOSS, TAKE PROFIT) für den automatisierten Handel, um das Risiko zu managen.

# ### 6. Test und Optimierung
# - **Überprüfung und Feinabstimmung**
#   - Durchführung von Tests der gesamten Pipeline in einer simulierten oder sicheren Umgebung.
#   - Feinabstimmung der Parameter und der Logik der Handelsstrategie basierend auf den Testergebnissen.
#   - Planung für regelmäßige Überprüfungen und Updates, um die Effektivität der Strategie zu erhalten oder zu verbessern.

In [ ]:


# add function to select between different cryptos. and then two charts below change!
# also add function for email notification? Full moon new moon maybe?

# also add diagram for support resistance!
# maybe bring this on own page? where the crypto dependent charts are that can be selected?

# # # start - plot volume bubble # # #
bubble_size = st.slider("Bubble Size", min_value=0, max_value=100, value=30)

# Normalizing the 'Volume'
xusd_data["norm_volume"] = (xusd_data["Volume"] - xusd_data["Volume"].min()) / (
    xusd_data["Volume"].max() - xusd_data["Volume"].min()
)
# Calculate daily price change
xusd_data["daily_change"] = xusd_data["Last"].diff()
# Assign colors based on positive or negative change
xusd_data["color"] = np.where(xusd_data["daily_change"] > 0, "green", "red")
# Create figure with secondary y-axis
fig_vol_bubble = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig_vol_bubble.add_trace(
    go.Bar(
        x=xusd_data.index,
        y=xusd_data["Volume"],
        name="Volume",
        marker=dict(
            color="black",
            opacity=0.5,
        ),
    ),
    secondary_y=True,
)

fig_vol_bubble.add_trace(
    go.Scatter(
        x=xusd_data.index,
        y=xusd_data["Last"],
        mode="markers+lines",
        marker=dict(
            size=xusd_data["norm_volume"] * bubble_size,  # Scale marker size
            color=xusd_data["color"],  # Assign color based on 'daily_change'
        ),
        name="Last",
        line=dict(color="black"),
    ),
    secondary_y=False,
)

fig_vol_bubble.update_layout(
    xaxis_title="Date",
    yaxis_title="Last",
    yaxis2_title="Volume",
    xaxis_rangeslider_visible=False,
    title=f"{selected_crypto} Bubble Volume Chart {datasource.split('/')[0]}",
    autosize=False,
    width=int(1400 / 1),
    height=int(800 / 1),
)
st.plotly_chart(fig_vol_bubble)
# # # end - plot volume bubble # # #


# # # start - newmoon fullmoon chart # # #

# TODO change it like the Volume Support chart so i can zoom in and have nice candles!

# take the xusd_data index and iterate over it to get the moon phase for each day
# the make a new df out of it called df_moon_phase
# Initialize an empty list to hold the rows.
list_of_rows = []

# Loop to populate the list_of_rows
for i in xusd_data.index:
    list_of_rows.append([i, moon.phase(i)])

# Convert the list of rows to a DataFrame.
df_moon_phase = pd.DataFrame(list_of_rows, columns=["date", "moon_phase"])


df_moon_phase["moon_phase"] = df_moon_phase["moon_phase"].fillna(0).astype(int)
df_moon_phase["date"] = pd.to_datetime(df_moon_phase["date"])
df_moon_phase["date"] = df_moon_phase["date"].dt.date
df_moon_phase = df_moon_phase.set_index("date")

# check the relationship between moon phase and xusd_data
# merge sp500 with moon phase
xusd_data_and_moon_phase = pd.merge(
    xusd_data, df_moon_phase, left_index=True, right_index=True, how="left"
)
xusd_data_and_moon_phase = xusd_data_and_moon_phase.dropna()

# 0 .. 6.99	New moon
# 7 .. 13.99	First quarter
# 14 .. 20.99	Full moon
# 21 .. 27.99	Last quarter
# create a new column called moon_phase_category and fill it with the moon_phase_category
xusd_data_and_moon_phase["moon_phase_category"] = ""
xusd_data_and_moon_phase.loc[
    (xusd_data_and_moon_phase["moon_phase"] >= 0)
    & (xusd_data_and_moon_phase["moon_phase"] <= 6.99),
    "moon_phase_category",
] = 0
xusd_data_and_moon_phase.loc[
    (xusd_data_and_moon_phase["moon_phase"] >= 7)
    & (xusd_data_and_moon_phase["moon_phase"] <= 13.99),
    "moon_phase_category",
] = 7
xusd_data_and_moon_phase.loc[
    (xusd_data_and_moon_phase["moon_phase"] >= 14)
    & (xusd_data_and_moon_phase["moon_phase"] <= 20.99),
    "moon_phase_category",
] = 14
xusd_data_and_moon_phase.loc[
    (xusd_data_and_moon_phase["moon_phase"] >= 21)
    & (xusd_data_and_moon_phase["moon_phase"] <= 27.99),
    "moon_phase_category",
] = 21


# make a new plot and plot the daily_price_change of btc by time on the x-axis
# fig_btc_moon = make_subplots(specs=[[{"secondary_y": True}]])
# Add the scatter plot for the 'Last' data
fig_btc_moon = go.Figure(
    data=[
        go.Candlestick(
            x=xusd_data_and_moon_phase.index,
            open=xusd_data_and_moon_phase["First"],
            high=xusd_data_and_moon_phase["High"],
            low=xusd_data_and_moon_phase["Low"],
            close=xusd_data_and_moon_phase["Last"],
        )
    ]
)
# Create empty lists to hold the x, y, and text values
x_new_moon = []
x_full_moon = []
y_new_moon = []
y_full_moon = []
text_new_moon = []
text_full_moon = []

last_moon_phase = None
# Loop over the dataframe
for i in range(len(xusd_data_and_moon_phase)):
    current_moon_phase = xusd_data_and_moon_phase["moon_phase_category"].iloc[i]
    if current_moon_phase != last_moon_phase:
        if current_moon_phase == 0:
            # Add the date to the x list, 0.9 to the y list, and the hover text to the text list
            x_new_moon.append(xusd_data_and_moon_phase.index[i])
            y_new_moon.append(xusd_data_and_moon_phase.Last[i] * 1.2)
            text_new_moon.append(f"New Moon: {xusd_data_and_moon_phase.index[i]}")
        elif current_moon_phase == 14:
            # Add the date to the x list, 0 to the y list, and the hover text to the text list
            x_full_moon.append(xusd_data_and_moon_phase.index[i])
            y_full_moon.append(xusd_data_and_moon_phase.Last[i] * 0.8)
            text_full_moon.append(f"Full Moon: {xusd_data_and_moon_phase.index[i]}")
        last_moon_phase = current_moon_phase

# Add a scatter plot to the figure for the new moon points
fig_btc_moon.add_trace(
    go.Scatter(
        x=x_new_moon,
        y=y_new_moon,
        mode="markers",
        marker=dict(symbol="circle", size=10, color="gray"),
        text=text_new_moon,
        hoverinfo="text",
        showlegend=False,
    )
)

# Add a scatter plot to the figure for the full moon points
fig_btc_moon.add_trace(
    go.Scatter(
        x=x_full_moon,
        y=y_full_moon,
        mode="markers",
        marker=dict(symbol="circle", size=10, color="yellow"),
        text=text_full_moon,
        hoverinfo="text",
        showlegend=False,
    )
)


fig_btc_moon.add_trace(
    go.Scatter(
        x=xusd_data_and_moon_phase.index,
        y=xusd_data_and_moon_phase.moon_phase,
        mode="lines",
        hoverinfo="text+y",  # It will show the hover text and y-value
        hovertext=xusd_data_and_moon_phase.moon_phase.apply(
            lambda x: f"Moon Phase: {x}"
        ),  # Showing moon phase value as hover text
        yaxis="y2",  # Specifies the usage of secondary y-axis
    )
)

# Update the layout to show the secondary y-axis on the right and other customizations
fig_btc_moon.update_layout(
    xaxis_title="Date",
    yaxis_title="Last",
    yaxis2=dict(
        title="Moon Phase",
        overlaying="y",  # Ensures it overlays on the primary y-axis
        side="right",  # Positioning the y-axis on the right
    ),
    title=f"{selected_crypto} moon Chart {datasource.split('/')[0]}",
    autosize=False,
    width=int(1400 / 1),
    height=int(800 / 1),
    xaxis_rangeslider_visible=False,
)

st.plotly_chart(fig_btc_moon)
# # # end - plot volume bubble # # #


# # # # start - plot Daily Volume Support Resistance Zones # # #
# num_zones = st.number_input("Enter the number of zones", value=12)
# # num_zones_int = int(num_zones)

# # Calculate frequencies of prices
# vol_profile = xusd_data_and_moon_phase["Last"].value_counts().nlargest(num_zones)
# print(vol_profile)

# # Define support and resistance levels
# support_levels = vol_profile.index.sort_values()
# resistance_levels = support_levels[1:]

# # Normalize volumes to range 1-5 for line widths
# vol_normalized = (vol_profile - vol_profile.min()) / (
#     vol_profile.max() - vol_profile.min()
# ) * 4 + 1

# # Plot price and volume profile (support and resistance levels)
# fig_volume_sup_res = go.Figure(
#     data=[
#         go.Candlestick(
#             x=xusd_data_and_moon_phase.index,
#             open=xusd_data_and_moon_phase["First"],
#             high=xusd_data_and_moon_phase["High"],
#             low=xusd_data_and_moon_phase["Low"],
#             close=xusd_data_and_moon_phase["Last"],
#         )
#     ]
# )

# # Add support and resistance lines
# for level in support_levels:
#     width = vol_normalized[level]
#     fig_volume_sup_res.add_trace(
#         go.Scatter(
#             x=xusd_data_and_moon_phase.index,
#             y=[level] * len(xusd_data_and_moon_phase.index),
#             mode="lines",
#             line=dict(width=width),
#             name=f"Vol: {vol_profile[level]}",
#         )
#     )

# fig_volume_sup_res.update_layout(
#     xaxis_title="Date",
#     yaxis_title="Last",
#     # xaxis_rangeslider_visible=True,
#     title=f"{selected_crypto} Daily Volume Support Resistance Zones {datasource.split('/')[0]}",
#     autosize=False,
#     width=int(1400 / 1),
#     height=int(800 / 1),
#     xaxis_rangeslider_visible=False,
# )

# st.plotly_chart(fig_volume_sup_res)
# # # # end - plot Daily Volume Support Resistance Zones # # #


# # # start - slider plot Daily Volume Support Resistance Zones # # #
# Create columns for layout
col1, col2, col3 = st.columns(3)
# Create a slider to select the number of days to go back
with col1:
    days_to_include = st.slider(
        "Days to consider:",
        min_value=5,
        max_value=len(xusd_data_and_moon_phase),
        value=len(xusd_data_and_moon_phase),
    )

with col2:
    num_bars = st.number_input("Bars:", value=12)

with col3:
    bin_percentage = st.number_input("Bin distance percentage:", value=10)

# Filter the data based on the days selected
xusd_data_and_moon_phase = xusd_data_and_moon_phase.iloc[-days_to_include:]

xusd_data_and_moon_phase["Bin_Width"] = (
    xusd_data_and_moon_phase["Last"] * bin_percentage / 100
)
bins = [
    xusd_data_and_moon_phase["Last"].iloc[0]
    - xusd_data_and_moon_phase["Bin_Width"].iloc[0]
]
for i in range(1, len(xusd_data_and_moon_phase)):
    bins.append(
        bins[-1]
        + (
            xusd_data_and_moon_phase["Bin_Width"].iloc[i]
            + xusd_data_and_moon_phase["Bin_Width"].iloc[i - 1]
        )
        / 2
    )

xusd_data_and_moon_phase["Bin"] = pd.cut(
    xusd_data_and_moon_phase["Last"], bins, labels=(bins[:-1]), include_lowest=True
)
binned_volume = xusd_data_and_moon_phase.groupby("Bin").size()
top_bins = binned_volume.nlargest(num_bars)

vol_normalized = (top_bins - top_bins.min()) / (top_bins.max() - top_bins.min()) * 4 + 1

fig_volume_sup_res_slider = go.Figure(
    data=[
        go.Candlestick(
            x=xusd_data_and_moon_phase.index,
            open=xusd_data_and_moon_phase["First"],
            high=xusd_data_and_moon_phase["High"],
            low=xusd_data_and_moon_phase["Low"],
            close=xusd_data_and_moon_phase["Last"],
        )
    ]
)

for level, vol in top_bins.items():
    width = vol_normalized[level]
    fig_volume_sup_res_slider.add_trace(
        go.Scatter(
            x=xusd_data_and_moon_phase.index,
            y=[level] * len(xusd_data_and_moon_phase.index),
            mode="lines",
            line=dict(width=width),
            name=f"Vol: {vol}",
        )
    )

fig_volume_sup_res_slider.update_layout(
    xaxis_title="Date",
    yaxis_title="Last",
    # xaxis_rangeslider_visible=True,
    title=f"{selected_crypto} Daily Volume Support Resistance Zones {datasource.split('/')[0]}",
    autosize=False,
    width=int(1400 / 1),
    height=int(800 / 1),
    xaxis_rangeslider_visible=False,
)

st.plotly_chart(fig_volume_sup_res_slider)
# # # end - slider plot Daily Volume Support Resistance Zones # # #

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)